# Gassmann Fluid Substitution

This notebook is based [on the original by Hadyan Pratama](https://github.com/ezygeo-ai/Fluid-Substitution), which is licenced under the MIT licence.

Original: Copyright (C) 2020 ezygeo.com, Hadyan Pratama (hadyan_pratama@rocketmail.com)
Modified: Matt Hall, 2021. Same licence.

I am going to check the result using `bruges`.

In [1]:
import bruges as bg

This Notebook requires bruges 0.4.0 or greater.

In [2]:
bg.__version__

'0.4.1'

In [3]:
import numpy as np

## Data

The original notebook had cgs units. I'm going to do everything in SI because science.

In [4]:
# Converting to SI units.
Vp = 3.349 * 1000
Vs = 1.836 * 1000

So = 0.7  # Initial condition.
Sw = 1    # Subbed case.

Por = 0.3

Fq = 0.7
Fc = 0.3
Kq = 36 * 1e9
Kc = 75 * 1e9
Ko = 1.6 * 1e9
Kw = 2.83 * 1e9

Rho = 2.13 * 1000
Rhoo = 0.8 * 1000
Rhow = 1 * 1000

## Define Formulae

The ezygeo functions all used rounding on the outputs... I have removed this 'feature'.

#### 1. K Saturated Initial

_Biot (1956)_

<img src="https://github.com/ezygeo-ai/Fluid-Substitution/blob/master/img/biot.png?raw=true" height=50% width=50%>

In [5]:
def msat(vs, rho):
    '''
    Miu saturated (C) hpratama 2020
    
    Input:
    vs = Shear Velocity (Km/s)
    rho = Density (g/cc)
  
    '''
    return rho*(vs**2)

I'm going to check all these formula against the equivalent in `bruges`, if it's in there.

In [6]:
assert msat(Vs, Rho) == bg.rockphysics.moduli.mu(vs=Vs, rho=Rho)

In [7]:
def ksat1(vp, vs, rho):
    '''
    K saturated (C) hpratama 2020
    
    Input:
    vp = P wave velocity (Km/s)
    vs = Shear wave Velocity (Km/s)
    rho = Density (g/cc)
  
    '''
    return ((vp**2)*rho) - 4/3*(msat(vs, rho))

In [8]:
assert ksat1(Vp, Vs, Rho) == bg.rockphysics.moduli.bulk(Vp, Vs, Rho)

#### 2. K Fluid 

_Batzle and Wang, 1992_

<img src="https://github.com/ezygeo-ai/Fluid-Substitution/blob/master/img/batzleandwang.png?raw=true" height=40% width=40%>

In [9]:
def kfl(sw, khc, kw):
    '''
    K Fluid (C) hpratama 2020
    
    Input:
    sw = Water Saturation
    khc = K Hydrocarbon
    kw = K Water
    
    '''
    return ((sw/kw) + ((1-sw)/khc))**-1

In [10]:
ezygeo = kfl(1 - So, Ko, Kw)
bruges = bg.rockphysics.fluids.wood(Kw, Ko, 1 - So)

assert ezygeo == bruges

#### 3. K Mineral

_Voight-Reuss-Hill Average_

<img src="https://github.com/ezygeo-ai/Fluid-Substitution/blob/master/img/voightreusshill.png?raw=true" height=50% width=50%>

In [11]:
def km(por, fq, fc, kq, kc):
    '''
    K mineral (K0) (C) hpratama 2020
    
    Input:
    por = Porosity
    fq = Quartz Fraction
    fc = Calcite Fraction
    kq = Quarzt Bulk
    kc = Calcite Bulk
    '''
    kv = ((fq)*kq) + ((fc)*kc) 
    kr = (((fq)/kq) + ((fc)/kc))**(-1)
    return (kv + kr)/2

In [12]:
ezygeo = km(Por, Fq, Fc, Kq, Kc)
bruges = bg.rockphysics.vrh(Kc, Kq, Fc)

assert ezygeo == bruges

#### 4. K Saturated Final 

_Mavko et al._

<img src="https://github.com/ezygeo-ai/Fluid-Substitution/blob/master/img/mavko.png?raw=true" height=50% width=50%>

In [13]:
def ksat2(ksat1, k0, kfl1, kfl2, por):
    '''
    K Saturation after fluid substitution (C) hpratama 2020
    
    Input:
    ksat1 = K Saturation before Fluid Substitution
    k0 = K Mineral
    kfl1 = K Fluid before Substitution
    kfl2 = K Fluid after Substitution
    por = Porosity
    
    '''
    a = (ksat1/(k0 - ksat1)) - (kfl1/(por*(k0 - kfl1))) + (kfl2/(por*(k0 - kfl2)))
    return (a*(k0))/(1+a)

In [14]:
# First Ezygeo's solution.
Ksat1 = ksat1(Vp, Vs, Rho)
Kfl1 = kfl((1-So), Ko, Kw)
Kfl2 = kfl(Sw, Ko, Kw)
K0 = km(Por, Fq, Fc, Kq, Kc)
Ksat2_ezygeo = ksat2(Ksat1, K0, Kfl1, Kfl2, Por)

# And using bruges.
Ksat1 = bg.rockphysics.moduli.bulk(Vp, Vs, Rho)
Kfl1 = bg.rockphysics.fluids.wood(Kw, Ko, 1 - So)
Kfl2 = bg.rockphysics.fluids.wood(Kw, Ko, Sw)
K0 = bg.rockphysics.vrh(Kc, Kq, Fc)
Ksat2_bruges = bg.rockphysics.fluidsub.avseth_gassmann(Ksat1, Kfl1, Kfl2, K0, Por)

np.testing.assert_almost_equal(Ksat2_ezygeo, Ksat2_bruges, decimal=4)

#### 5. Vp 

<img src="https://github.com/ezygeo-ai/Fluid-Substitution/blob/master/img/rho_2.png?raw=true" height=50% width=50%>

Compute fluid density:

In [15]:
def rhofluid(sw, so, rhoo, rhow):
    
    rho1 = ((1-so) * rhow) + (so * rhoo)
    rho2 = ((sw) * rhow) + ((1 - sw) * rhoo)
    return rho1, rho2

In [16]:
# Lines adapted from bruges source code.
rhofl = (1 - So) * Rhow + So * Rhoo
rhofl2 = 1 * Rhow

assert rhofluid(Sw, So, Rhoo, Rhow) == (rhofl, rhofl2)

Compute new rock density:

In [17]:
def rhosat(rho, por, rhow, sw, so, rhoo):
    rho1, rho2 = rhofluid(sw, so, rhoo, rhow)
    return rho + (por * (rho2 - rho1))

In [18]:
# Lines adapted from bruges source code.
rhog = (Rho - Por * rhofl) / (1 - Por)
rho2 = Por * rhofl2 + (1 - Por) * rhog

# ezygeo solution.
Rhosat = rhosat(Rho, Por, Rhow, Sw, So, Rhoo)

assert Rhosat == rho2

Compute new Vp:

In [19]:
def vp(ksat2, vs, rhosat):
    return ((ksat2 + (4/3*(msat(vs, rhosat)))) / (rhosat))**0.5

In [20]:
ezygeo = vp(Ksat2_ezygeo, Vs, Rhosat)
#                         ^^ Error: Vs(2) != Vs(1)

G = bg.rockphysics.moduli.mu(vs=Vs, rho=Rho)
bruges = bg.rockphysics.moduli.vp(bulk=Ksat2_bruges, mu=G, rho=rho2)

ezygeo, bruges

(3439.9002222704626, 3427.2441956687944)

## Calculate `ezygeo` result

To get new value of Vp, there is some variable that need to calculate.

* First we need to calculate K saturation before fluid subtitution. This K saturartion could be calculated from the old 

In [21]:
Ksat1 = ksat1(Vp, Vs, Rho)

* Then, we estimate the K fluid before and after fluid substitution

In [22]:
Kfl1 = kfl((1-So), Ko, Kw)
Kfl2 = kfl(Sw, Ko, Kw)  # = Kw

* After that, we evaluate K Mineral that can be estimate from Hill formula, this formula need Voight-Reuss Bounds calculation too. First we need to calculate K saturation before fluid subtitution.

In [23]:
K0 = km(Por, Fq, Fc, Kq, Kc)

* Here we calculate K saturated after fluid substitution

In [24]:
Ksat2 = ksat2(Ksat1, K0, Kfl1, Kfl2, Por)

* To calculate the new Vp, we need to calculate density saturation. After that we estimate new Vp with new K saturated after fluid substitution

In [25]:
Rhosat = rhosat(Rho, Por, Rhow, Sw, So, Rhoo)

In [26]:
Vp2 = vp(Ksat2, Vs, Rhosat)

In [27]:
print(Vp2)

3439.9002222704626


Done.

## Compute using Avseth's algorithm in `bruges`

In [28]:
from bruges.rockphysics.fluidsub import smith_gassmann, avseth_gassmann, avseth_fluidsub, smith_fluidsub

In [29]:
avseth_fluidsub(Vp, Vs, Rho,
                Por,
                rhof1=(1 - So) * Rhow + So * Rhoo,
                rhof2=Rhow,
                kmin=bg.rockphysics.vrh(Kc, Kq, Fc),
                kf1=bg.rockphysics.fluids.wood(Kw, Ko, 1-So),
                kf2=Kw,
               )

FluidSubResult(Vp=3427.2441956687944, Vs=1818.1619642311286, rho=2172.0)

## Compute using Smith's algorithm in `bruges`

In [30]:
smith_fluidsub(Vp, Vs, Rho,
               Por,
               Rhow, Rhoo,
               sw=1 - So, swnew=1,  # Fluid 1: Mix, Fluid 2: Brine
               kw=Kw,
               khc=Ko,
               kclay=Kc,
               kqtz=Kq,
               vclay=Fc,
               )

FluidSubResult(Vp=3427.244195668794, Vs=1818.1619642311286, rho=2172.0)

This is not the same as the _ezygeo_ result, I think because of the error of passing Vs to the `vp()` function.